In [ ]:
!pip install websocket-client

In [ ]:
import os
import json
import random
import requests

from ipyleaflet import Map, GeoJSON

if not os.path.exists('europe_110.geo.json'):
    url = 'https://github.com/jupyter-widgets/ipyleaflet/raw/master/examples/europe_110.geo.json'
    r = requests.get(url)
    with open('europe_110.geo.json', 'w') as f:
        f.write(r.content.decode("utf-8"))

with open('europe_110.geo.json', 'r') as f:
    data = json.load(f)

def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

m = Map(center=(50.6252978589571, 0.34580993652344), zoom=3)

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
m.add_layer(geo_json)

m

In [23]:
import os
import json
import random
import requests

from ipyleaflet import Map, GeoJSON, TileLayer

url = "https://cartocdn-gusc.global.ssl.fastly.net/opmbuilder/api/v1/map/named/opm-mars-basemap-v0-1/all/{z}/{x}/{y}.png" 
provider = TileLayer(url=url, zoom=3, tms=False)

url = "http://s3-eu-west-1.amazonaws.com/whereonmars.cartodb.net/celestia_mars-shaded-16k_global/{z}/{x}/{y}.png" 
hirise = TileLayer(url=url, zoom=3,  tms=True)

m = Map(center=(-2.04694183792199, 354.505873462974), zoom=3)
#m.add_layer(provider)
#m.add_layer(hirise)
m

Map(center=[-2.04694183792199, 354.505873462974], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [19]:
import json
r = requests.get('http://localhost:4321/api/integration/geojson_latlon')
print(json.loads(r.content))
m.center = (-2.04694183792199, 354.505873462974)
m.zoom=18
r.content

{'geometries': [{'coordinates': [[-2.02335898044766, 354.490557290681, -2016.07439950911], [-2.05723682043148, 354.478648248411, -2017.44387354697]], 'type': 'LineString'}, {'coordinates': [[-2.05479945570688, 354.464955302623, -2025.89080756544], [-2.02424466627644, 354.484151073537, -2012.50608908448]], 'type': 'LineString'}, {'coordinates': [[-2.02353834234429, 354.482720954918, -2013.0173775663]], 'type': 'LineString'}], 'type': 'GeometryCollection'}


b'{\n  "geometries": [\n    {\n      "coordinates": [\n        [\n          -2.02335898044766,\n          354.490557290681,\n          -2016.07439950911\n        ],\n        [\n          -2.05723682043148,\n          354.478648248411,\n          -2017.44387354697\n        ]\n      ],\n      "type": "LineString"\n    },\n    {\n      "coordinates": [\n        [\n          -2.05479945570688,\n          354.464955302623,\n          -2025.89080756544\n        ],\n        [\n          -2.02424466627644,\n          354.484151073537,\n          -2012.50608908448\n        ]\n      ],\n      "type": "LineString"\n    },\n    {\n      "coordinates": [\n        [\n          -2.02353834234429,\n          354.482720954918,\n          -2013.0173775663\n        ]\n      ],\n      "type": "LineString"\n    }\n  ],\n  "type": "GeometryCollection"\n}'

In [24]:
import websocket
import _thread
import time
import json



def on_message(ws, message):
    global geo_json
    print("Jdsjd")
    r = json.loads(requests.get('http://localhost:4321/api/integration/geojson_latlon').content)
    f = open("F:\\pro3d\\PRo3D\\src\\Tests\\test.geo.json", 'r') 
    d = json.load(f)
    print("doit")
    print(r)
    m.center = (-2.04694183792199, 354.505873462974)
    geo_json2 = GeoJSON(
        data=d
    )
    m.add_layer(geo_json2)
    #m.remove_layer(geo_json)
    geo_json = geo_json2
    ws.send("blub")

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print("### closed ###")

def on_open(ws):
    print("Opened connection")



ws = websocket.WebSocketApp("ws://localhost:4321/api/integration/ws/geojson_xyz",
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

ws.run_forever()  

Opened connection
Jdsjd
doit
{'geometries': [{'coordinates': [[-2.02335898044766, 354.490557290681, -2016.07439950911], [-2.05723682043148, 354.478648248411, -2017.44387354697]], 'type': 'LineString'}, {'coordinates': [[-2.05479945570688, 354.464955302623, -2025.89080756544], [-2.02424466627644, 354.484151073537, -2012.50608908448]], 'type': 'LineString'}, {'coordinates': [[-2.02353834234429, 354.482720954918, -2013.0173775663]], 'type': 'LineString'}, {'coordinates': [[-2.03474829195083, 354.487364579821, -2013.23558926354], [-2.03251016210663, 354.482455813633, -2014.04769256488]], 'type': 'LineString'}, {'coordinates': [[-2.01923211082753, 354.474881627473, -2017.89458887325]], 'type': 'LineString'}, {'coordinates': [[-2.03866428835667, 354.478039722183, -2016.24412040495], [-2.03248568747887, 354.478962220847, -2014.29381061419]], 'type': 'LineString'}, {'coordinates': [[-2.02723984827085, 354.478288195918, -2012.42299444633]], 'type': 'LineString'}, {'coordinates': [[-2.0394738783